In [2]:
import ipywidgets as widgets
from IPython.display import display
from DataCollector import DataCollector
from Author import Author
from Document import Document
from RedditDocument import RedditDocument
from ArxivDocument import ArxivDocument
import datetime
import pandas as pd

def collect_data_and_display(subreddit, reddit_limit, arxiv_query_terms, arxiv_max_results):
    # Créer une instance de DataCollector avec les paramètres fournis
    collector = DataCollector(subreddit=subreddit_widget.value, limit=reddit_limit_widget.value,
                          query_terms=arxiv_query_terms_widget.value.split(','), max_results=arxiv_max_results_widget.value)

    # Collecter des données depuis Reddit
    collector.collect_reddit_data()

    # Collecter des données depuis ArXiv
    collector.collect_arxiv_data()
       
    collection = []
    for nature, doc in collector.docs_bruts:
        if nature == "ArXiv": 
            titre = doc["title"].replace('\n', '')  # On enlève les retours à la ligne
            try:
                authors = ", ".join([a["name"] for a in doc["author"]])  # On fait une liste d'auteurs, séparés par une virgule
            except:
                authors = doc["author"]["name"]  # Si l'auteur est seul, pas besoin de liste
            summary = doc["summary"].replace("\n", "")  # On enlève les retours à la ligne
            date = datetime.datetime.strptime(doc["published"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y/%m/%d")  # Formatage de la date en année/mois/jour avec librairie datetime

            doc_classe = Document(titre, authors, date, doc["id"], summary)  # Création du Document
            collection.append(doc_classe)  # Ajout du Document à la liste.

        elif nature == "Reddit":
            titre = doc.title.replace("\n", '')
            auteur = str(doc.author)
            date = datetime.datetime.fromtimestamp(doc.created).strftime("%Y/%m/%d")
            url = str(doc.url)
            texte = doc.selftext.replace("\n", "")
            doc_classe = Document(titre, auteur, date, url, texte)
            collection.append(doc_classe)
    
    authors = {}
    aut2id = {}
    num_auteurs_vus = 0
    
    for doc in collection:
        if doc.auteur not in aut2id:
            num_auteurs_vus += 1
            authors[num_auteurs_vus] = Author(doc.auteur)
            aut2id[doc.auteur] = num_auteurs_vus

        authors[aut2id[doc.auteur]].add(doc.texte) 
    
    corpus_data = {
        'Titre': [],
        'Auteur': [],
        'Date': [],
        'URL': [],
        'Texte': []
    }
    
    for doc in collection:
        corpus_data['Titre'].append(doc.titre)
        corpus_data['Auteur'].append(doc.auteur)
        corpus_data['Date'].append(doc.date)
        corpus_data['URL'].append(doc.url)
        corpus_data['Texte'].append(doc.texte)
        
    corpus_df = pd.DataFrame(corpus_data)
    corpus_df.to_csv('corpus_data.csv', sep='\t', index=False)
    
    message = "le fichier corpus_data.csv est crée avec {} auteurs et {} documents."
    print(message.format(len(authors), len(collection)))

# Créer les widgets pour le formulaire
subreddit_widget = widgets.Text(value='corona', description='Subreddit:')
reddit_limit_widget = widgets.IntSlider(value=100, min=1, max=500, step=1, description='Reddit Limit:')
arxiv_query_terms_widget = widgets.Text(value='corona', description='ArXiv Query Terms:')
arxiv_max_results_widget = widgets.IntSlider(value=100, min=1, max=500, step=1, description='ArXiv Max Results:')

# Créer le bouton pour déclencher la collecte de données
button = widgets.Button(description='Collect and Display Data')

# Définir la fonction à exécuter lors du clic sur le bouton
def on_button_click(b):
    collect_data_and_display(subreddit_widget.value, reddit_limit_widget.value,
                              arxiv_query_terms_widget.value.split(','), arxiv_max_results_widget.value)

# Associer la fonction au clic sur le bouton
button.on_click(on_button_click)

# Afficher les widgets
display(subreddit_widget, reddit_limit_widget, arxiv_query_terms_widget, arxiv_max_results_widget, button)


Text(value='corona', description='Subreddit:')

IntSlider(value=100, description='Reddit Limit:', max=500, min=1)

Text(value='corona', description='ArXiv Query Terms:')

IntSlider(value=100, description='ArXiv Max Results:', max=500, min=1)

Button(description='Collect and Display Data', style=ButtonStyle())

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



le fichier corpus_data.csv est crée avec 1 auteurs et 1 documents.
